In [9]:
import numpy as np
import pandas as pd
import math
import itertools
import datetime
import operator
import csv

In [10]:
def countPath(path, arr, rozmiar,permutacja):
    #print('\npath:',path)
    #print('arr:',arr)
    #print('rozmiar',rozmiar)
#     if type(path) is tuple:
#         path = path[0]
    new_path=0
    new_path+=arr[0][path[0]]
    try:
        for step in range(rozmiar-2):
            new_path+=arr[path[step]][path[step+1]]
    except:
        print('\nnew_path+=arr[path[step]][path[step+1]]::')
        print("new_path:",new_path)
        print('path:',path)
        print('permutacja:',permutacja)
        print('path.size:',path.size)
        print('step:',step)
        print('rozmiar-2:', rozmiar-2)
        print('arr.shape:',arr.shape)
        
    new_path+=arr[path[rozmiar-2]][0]
    return new_path

In [11]:
def wczytanieWynikow():
    the_best_solutions = pd.read_csv('Dane\\wyniki.csv', header=None, sep=';', index_col=0, names=['costam','best_length'])
    return the_best_solutions

In [12]:
def wczytanieMacierzy(rozmiar):
    file = 'Dane/data{}.txt'.format(rozmiar)
    df = pd.read_csv(file, nrows=2, header=None)
    name = df[df.index==0].values[0][0]
    n = int(df[df.index==1].values[0][0])

    df = pd.read_csv(file, skiprows=2, sep='\s+', header=None)
    arr = df.values
    return arr

In [13]:
def inicjalizacja_poczatkowych_wartosci(arr,n):
    liczba_iteracji = n
    najkrotsza_sciezka = np.random.permutation(range(1,n))
    najlepszy_wynik = countPath(najkrotsza_sciezka, arr, n,10)
    for i in range(liczba_iteracji):
        losowa_sciezka = np.random.permutation(range(1,n))
        wynik = countPath(losowa_sciezka, arr, n,10)
        if wynik < najlepszy_wynik:
            najlepszy_wynik = wynik
            najkrotsza_sciezka = losowa_sciezka
        
    return najkrotsza_sciezka, najlepszy_wynik

In [14]:
def zaburzonePozycje(n):
    a,b = 0,0
    while a == b:
        a,b = np.random.randint(1, high=n-1, size=2)
    return a,b

In [15]:
def swap_sciezka(zaburzona_sciezka,a,b):
    zaburzona_sciezka[a], zaburzona_sciezka[b] = zaburzona_sciezka[b], zaburzona_sciezka[a]
    return zaburzona_sciezka

In [16]:
def insert_sciezka(zaburzona_sciezka,a,b,n):
    if np.random.rand() > 0.5:
        a, b = b, a
    while True:
        if b+1==a:
                a,b = zaburzonePozycje(n)
        else:
            break
    
    zaburzona_sciezka = np.insert(zaburzona_sciezka, a, zaburzona_sciezka[b])
    if a<b:
        zaburzona_sciezka = np.hstack((zaburzona_sciezka[:b+1],zaburzona_sciezka[b+2:]))
    else:
        zaburzona_sciezka = np.hstack((zaburzona_sciezka[:b],zaburzona_sciezka[b+1:]))

    return zaburzona_sciezka, a, b

In [17]:
def invert_sciezka(zaburzona_sciezka,a,b):
    if b<a:
        a, b = b, a
    while a<b:
        zaburzona_sciezka[a], zaburzona_sciezka[b] = zaburzona_sciezka[b], zaburzona_sciezka[a]
        a += 1
        b -= 1
        
    return zaburzona_sciezka

In [18]:
def sprawdzenie_listy_tabu(lista_tabu, iteratool, zaburzona_sciezka):
    szukany_tuple = ()
    a=zaburzona_sciezka[iteratool[1]]
    b=zaburzona_sciezka[iteratool[0]]
    szukany_tuple = tuple((a,b))
    return szukany_tuple in lista_tabu

In [19]:
def sprawdzenie_kryterium_aspiracji1(najkrotsza_dlugosc, dlugosc_zaburzonej_sciezki):
    return dlugosc_zaburzonej_sciezki < najkrotsza_dlugosc

In [20]:
def sprawdzenie_kryterium_aspiracji2(dlugosc, dlugosc_zaburzonej_sciezki):
    return dlugosc_zaburzonej_sciezki < dlugosc

In [21]:
def sortuj_wg_wartosci(dictt): 
    lst = sorted(dictt.items(), key=lambda kv: kv[1])
    res_dct = {}
    for i in range(len(lst)):
        res_dct[lst[i][0]] = lst[i][1]
    return res_dct 

In [22]:
def obliczenia(rozmiary, liczba_powtorzen, csvwriter, struct): 
    jakosc_wynikow = []
    the_best_solutions = wczytanieWynikow()
    
    for nr, rozmiar in enumerate(rozmiary): #5 rozmiarow
        struct['rozmiar'] = rozmiar
        slownik_master = [int(np.sqrt(rozmiar)),rozmiar,rozmiar*rozmiar]
        lista_tabu = [7,10,rozmiar] 
        ilosc_powtorzen = liczba_powtorzen[nr]
        jakosc_wyniku = 0.0
        nazwa = 'data{}'.format(rozmiar)
        print('\nRozmiar:',rozmiar, end=', ')
        print('rozmiar_slownika_master:', end=' ')
        jakosc_master = []
        for rozmiar_slownika_master in slownik_master: #5*3=15
            print(rozmiar_slownika_master, end=',')
            struct['rozmiar_slownika_master'] = rozmiar_slownika_master
            jakosc_lista_tabu = []
            for dlugosc_listy_tabu in lista_tabu: #15*3=45
                struct['dlugosc_listy_tabu'] = dlugosc_listy_tabu
                jakosc_kryterium_aspiracji = []
                for kryterium_aspiracji in range(1,3): #45*2=90
                    struct['kryterium_aspiracji'] = kryterium_aspiracji
                    jakosc_permutacji = []
                    for nr_permutacji in range(3): #90*3=270
                        struct['nr_permutacji'] = nr_permutacji
                        for powtorzenie in range(ilosc_powtorzen): #270*10=2700
                            arr = wczytanieMacierzy(rozmiar)
                            #2700*7s=18900=316min=5,25h
                            wynik = obliczenieCzasuWyniku(arr, rozmiar, nr_permutacji, kryterium_aspiracji, rozmiar_slownika_master, dlugosc_listy_tabu)

                            nowa_jakosc_wyniku = the_best_solutions.loc[[nazwa],['best_length']].values[0,0]/wynik
                            if nowa_jakosc_wyniku > jakosc_wyniku:
                                jakosc_wyniku = nowa_jakosc_wyniku
                            
                        struct['jakosc_wyniku'] = jakosc_wyniku
                        csvwriter.writerow(struct)
                        jakosc_permutacji.append(jakosc_wyniku)
                    jakosc_kryterium_aspiracji.append(jakosc_permutacji)
                jakosc_lista_tabu.append(jakosc_kryterium_aspiracji)
            jakosc_master.append(jakosc_lista_tabu)        
        jakosc_wynikow.append(jakosc_master)
    
    return jakosc_wynikow

In [24]:
def obliczenieCzasuWyniku(arr, n, permutacja, kryterium_aspiracji, rozmiar_slownika_master, dlugosc_listy_tabu):
    start = datetime.datetime.now()
    poczatkowa_sciezka, najkrotsza_dlugosc = inicjalizacja_poczatkowych_wartosci(arr,n)
    i = 0
    max_czas = 7.0
    sciezka = poczatkowa_sciezka
    najlepsza_sciezka = sciezka
    lista_tabu = {}
    slownik_master = {}
    master_wartosc_ostatniego=0
        
    duration = datetime.datetime.now() - start
    
    while duration.total_seconds() < max_czas:
        dlugosc = countPath(sciezka, arr, n,20)
        przejscie_zakazane = ()
        slownik_sasiadow = {}
        if len(slownik_master) == 0:
            for iteratool in itertools.combinations(range(0, n-1), 2):

                #Zaburzenie sciezki:
                a = iteratool[0] # a<b zawsze
                b = iteratool[1]
                zaburzona_sciezka = sciezka.copy()
                
                if permutacja == 0:
                    zaburzona_sciezka = swap_sciezka(zaburzona_sciezka,a,b)

                elif permutacja == 1:
                    #print('zaburzona_sciezka',zaburzona_sciezka, 'a',a,'b',b,'n',n)
                    zaburzona_sciezka,a,b = insert_sciezka(zaburzona_sciezka,a,b,n)
                    #print('zaburzona_sciezka',zaburzona_sciezka, 'a',a,'b',b,'n',n)
                    #print('--------------------------------------------------------------')

                else:
                    zaburzona_sciezka = invert_sciezka(zaburzona_sciezka,a,b)


                dlugosc_zaburzonej_sciezki = countPath(zaburzona_sciezka, arr, n,permutacja)
                slownik_sasiadow[iteratool] = dlugosc_zaburzonej_sciezki

            slownik_sasiadow = sortuj_wg_wartosci(slownik_sasiadow)
            master_wartosc_ostatniego = 0
            tmp = 0
            for iteratool, dlugosc_zaburzonej_sciezki in slownik_sasiadow.items():
                slownik_master[iteratool] = dlugosc_zaburzonej_sciezki
                tmp +=1
                master_wartosc_ostatniego = dlugosc_zaburzonej_sciezki
                if tmp == rozmiar_slownika_master:
                    break
        
        else:
            for iteratool in slownik_master.keys():

                #Zaburzenie sciezki:
                a = iteratool[0] # a<b zawsze
                b = iteratool[1]
                zaburzona_sciezka = sciezka.copy()
                
                if permutacja == 0:
                    zaburzona_sciezka = swap_sciezka(zaburzona_sciezka,a,b)
                elif permutacja == 1:
                    zaburzona_sciezka,a,b = insert_sciezka(zaburzona_sciezka,a,b,n)
                else:
                    zaburzona_sciezka = invert_sciezka(zaburzona_sciezka,a,b)

                dlugosc_zaburzonej_sciezki = countPath(zaburzona_sciezka, arr, n, 132)
                slownik_sasiadow[iteratool] = dlugosc_zaburzonej_sciezki

            slownik_sasiadow = sortuj_wg_wartosci(slownik_sasiadow)
            slownik_master = slownik_sasiadow
            
        for iteratool, dlugosc_zaburzonej_sciezki in list(slownik_master.items()):
            slownik_master.pop(iteratool)
            
            if dlugosc_zaburzonej_sciezki > master_wartosc_ostatniego:
                slownik_master = {}
                break
            
            czy_przejscie_na_liscie_tabu = sprawdzenie_listy_tabu(lista_tabu, iteratool, zaburzona_sciezka)
            czy_spelnia_kryterium_aspiracji = 0
            
            if czy_przejscie_na_liscie_tabu:
                if kryterium_aspiracji == 1:
                    czy_spelnia_kryterium_aspiracji = sprawdzenie_kryterium_aspiracji1(najkrotsza_dlugosc, dlugosc_zaburzonej_sciezki)
                else:
                    czy_spelnia_kryterium_aspiracji = sprawdzenie_kryterium_aspiracji2(dlugosc, dlugosc_zaburzonej_sciezki)
            
            if ~czy_przejscie_na_liscie_tabu or czy_spelnia_kryterium_aspiracji:
                dlugosc = dlugosc_zaburzonej_sciezki
                sciezka = zaburzona_sciezka
                przejscie_zakazane = iteratool
                break
        
        if dlugosc < najkrotsza_dlugosc:
            najkrotsza_dlugosc = dlugosc
            najlepsza_sciezka = sciezka
        
        for klucz in list(lista_tabu):
            if lista_tabu[klucz] == 1:
                lista_tabu.pop(klucz)
            else:
                lista_tabu[klucz] -= 1

        if len(przejscie_zakazane):
            a = sciezka[przejscie_zakazane[1]]
            b = sciezka[przejscie_zakazane[0]]
            przejscie_zakazane = tuple((a,b))
            lista_tabu[przejscie_zakazane] = dlugosc_listy_tabu
        
        i += 1        
        duration = datetime.datetime.now() - start
    
#     print('iteration:',i)
#     print('duration:',duration.total_seconds(), '\n')
    
    return najkrotsza_dlugosc

In [16]:
struct = {
            'rozmiar' : 0,
            'rozmiar_slownika_master' : 0,
            'dlugosc_listy_tabu' : 0,
            'kryterium_aspiracji' : 0,
            'nr_permutacji' : 0,
            'jakosc_wyniku' : 0
        }
with open('tabu_search.csv','w') as csvfile:
    fieldnames = ['rozmiar', 'rozmiar_slownika_master', 'dlugosc_listy_tabu', 'kryterium_aspiracji', 'nr_permutacji', 'jakosc_wyniku']
    csvwriter = csv.DictWriter(csvfile, fieldnames=fieldnames)
    csvwriter.writeheader()

    rozmiary = [21, 48, 71, 120, 323]
    liczba_powtorzen = [10, 10, 10, 10, 10]
    #rozmiary = [71]
    #liczba_powtorzen = [1]
    jakosc_wynikow = obliczenia(rozmiary, liczba_powtorzen, csvwriter, struct)


Rozmiar: 21, rozmiar_slownika_master: 4,21,441,
Rozmiar: 48, rozmiar_slownika_master: 6,48,2304,
Rozmiar: 71, rozmiar_slownika_master: 8,71,5041,
Rozmiar: 120, rozmiar_slownika_master: 10,120,14400,
Rozmiar: 323, rozmiar_slownika_master: 17,323,104329,

In [17]:
df = pd.read_csv('tabu_search.csv')
df

,rozmiar,rozmiar_slownika_master,dlugosc_listy_tabu,kryterium_aspiracji,nr_permutacji,jakosc_wyniku
0,21,4,7,1,0,0.769034
1,21,4,7,1,1,0.833949
2,21,4,7,1,2,0.909916
3,21,4,7,2,0,0.909916
4,21,4,7,2,1,0.909916
...,...,...,...,...,...,...
265,323,104329,323,1,1,0.242546
266,323,104329,323,1,2,0.242546
267,323,104329,323,2,0,0.242546
268,323,104329,323,2,1,0.242546


In [8]:
def obliczenieCzasuWyniku(arr, n, permutacja, kryterium_aspiracji, rozmiar_slownika_master, dlugosc_listy_tabu):
    start = datetime.datetime.now()
    poczatkowa_sciezka, najkrotsza_dlugosc = inicjalizacja_poczatkowych_wartosci(arr,n)
    i = 0
    max_czas = 7.0
    sciezka = poczatkowa_sciezka
    najlepsza_sciezka = sciezka
    lista_tabu = {}
    slownik_master = {}
    master_wartosc_ostatniego=0
        
    duration = datetime.datetime.now() - start
    
    while duration.total_seconds() < max_czas:
        dlugosc = countPath(sciezka, arr, n,20)
        przejscie_zakazane = ()
        slownik_sasiadow = {}
        
        if len(slownik_master) == 0:
            for iteratool in itertools.combinations(range(0, n-1), 2):
                a = iteratool[0] # a<b zawsze
                b = iteratool[1]
                zaburzona_sciezka = sciezka.copy()
                
                if permutacja == 0:
                    zaburzona_sciezka = swap_sciezka(zaburzona_sciezka,a,b)

                elif permutacja == 1:
                    zaburzona_sciezka,a,b = insert_sciezka(zaburzona_sciezka,a,b,n)
                    
                else:
                    zaburzona_sciezka = invert_sciezka(zaburzona_sciezka,a,b)

                dlugosc_zaburzonej_sciezki = countPath(zaburzona_sciezka, arr, n,permutacja)
                slownik_sasiadow[iteratool] = dlugosc_zaburzonej_sciezki

            slownik_sasiadow = sortuj_wg_wartosci(slownik_sasiadow)
            master_wartosc_ostatniego = 0
            tmp = 0
            for iteratool, dlugosc_zaburzonej_sciezki in slownik_sasiadow.items():
                slownik_master[iteratool] = dlugosc_zaburzonej_sciezki
                tmp +=1
                master_wartosc_ostatniego = dlugosc_zaburzonej_sciezki
                if tmp == rozmiar_slownika_master:
                    break
        
        else:
            for iteratool in slownik_master.keys():
                a = iteratool[0] # a<b zawsze
                b = iteratool[1]
                zaburzona_sciezka = sciezka.copy()
                
                if permutacja == 0:
                    zaburzona_sciezka = swap_sciezka(zaburzona_sciezka,a,b)
                elif permutacja == 1:
                    zaburzona_sciezka,a,b = insert_sciezka(zaburzona_sciezka,a,b,n)
                else:
                    zaburzona_sciezka = invert_sciezka(zaburzona_sciezka,a,b)

                dlugosc_zaburzonej_sciezki = countPath(zaburzona_sciezka, arr, n, 132)
                slownik_sasiadow[iteratool] = dlugosc_zaburzonej_sciezki

            slownik_sasiadow = sortuj_wg_wartosci(slownik_sasiadow)
            slownik_master = slownik_sasiadow
            
        for iteratool, dlugosc_zaburzonej_sciezki in list(slownik_master.items()):
            slownik_master.pop(iteratool)
            
            if dlugosc_zaburzonej_sciezki > master_wartosc_ostatniego:
                slownik_master = {}
                break
            
            czy_przejscie_na_liscie_tabu = sprawdzenie_listy_tabu(lista_tabu, iteratool, zaburzona_sciezka)
            czy_spelnia_kryterium_aspiracji = 0
            
            if czy_przejscie_na_liscie_tabu:
                if kryterium_aspiracji == 1:
                    czy_spelnia_kryterium_aspiracji = sprawdzenie_kryterium_aspiracji1(najkrotsza_dlugosc, dlugosc_zaburzonej_sciezki)
                else:
                    czy_spelnia_kryterium_aspiracji = sprawdzenie_kryterium_aspiracji2(dlugosc, dlugosc_zaburzonej_sciezki)
            
            if ~czy_przejscie_na_liscie_tabu or czy_spelnia_kryterium_aspiracji:
                dlugosc = dlugosc_zaburzonej_sciezki
                sciezka = zaburzona_sciezka
                przejscie_zakazane = iteratool
                break
        
        if dlugosc < najkrotsza_dlugosc:
            najkrotsza_dlugosc = dlugosc
            najlepsza_sciezka = sciezka
        
        for klucz in list(lista_tabu):
            if lista_tabu[klucz] == 1:
                lista_tabu.pop(klucz)
            else:
                lista_tabu[klucz] -= 1

        if len(przejscie_zakazane):
            a = sciezka[przejscie_zakazane[1]]
            b = sciezka[przejscie_zakazane[0]]
            przejscie_zakazane = tuple((a,b))
            lista_tabu[przejscie_zakazane] = dlugosc_listy_tabu
        
        i += 1        
        duration = datetime.datetime.now() - start

    return najkrotsza_dlugosc

(1, 8)